In [29]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

In [42]:
def build_dataset(chorales):
    min_len = len(min(chorales, key=len))
    print(min_len)
    
    def reshape(chorale):
        chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
        return chorale
    def trunc(chorale):
        return chorale[:min_len+1]
        
    def target(chorale):
        X = chorale[:-1]
        Y = chorale[-1]
        return X, Y
    
    ragged_chorales = tf.ragged.constant(chorales)
    dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
    dataset = dataset.map(reshape)
    dataset = dataset.map(trunc)
    dataset = dataset.map(target)
    dataset = dataset.batch(32)
#     for d in dataset:
#         tf.print(d)
#         break
    return dataset

        
train_dataset = build_dataset(train_np_list)
val_dataset = build_dataset(val_np_list)
test_dataset = build_dataset(test_np_list)
build_dataset(train_np_list)

100
128
128
100


<BatchDataset shapes: ((None, None, 1), (None, 1)), types: (tf.int32, tf.int32)>

In [41]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])
model.compile(loss='mse',
             optimizer=keras.optimizers.Nadam(learning_rate=0.01),
             metrics=[keras.metrics.mean_squared_error])
model.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
8/8 [==============================] - 1s 35ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 2/100
8/8 [==============================] - 0s 16ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 3/100
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 4/100
8/8 [==============================] - 0s 17ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 5/100
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 6/100
8/8 [==============================] - 0s 18ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - va